# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

pd.set_option('display.max_columns',None)

In [108]:
flight_df = pd.read_feather('data/v2_clean_flight')
flight_test_df = pd.read_feather('data/v1_clean_flight_test')
fuel_consumption_df = pd.read_feather('data/v1_clean_fuel_consumption')
passenger_df = pd.read_feather('data/v1_clean_passenger')

In [109]:
flight_df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_delay,taxi_out,taxi_in,crs_arr_time,arr_delay,crs_elapsed_time,distance,month,flight_duration,hour,state/country
0,2019-10-28,AA,AA_CODESHARE,AA,4377,YX,N115HQ,4377,10785,BTV,"Burlington, VT",14100,PHL,"Philadelphia, PA",543,-4.0,10.0,8.0,713,-15.0,90.0,335.0,10,short,7,VT
1,2018-06-04,DL,DL_CODESHARE,DL,5491,EV,N741EV,5491,12953,LGA,"New York, NY",11042,CLE,"Cleveland, OH",1415,-5.0,24.0,7.0,1603,-16.0,108.0,419.0,6,short,16,NY
2,2018-07-31,WN,WN,WN,562,WN,N479WN,562,13198,MCI,"Kansas City, MO",13487,MSP,"Minneapolis, MN",940,-4.0,9.0,5.0,1055,-8.0,75.0,393.0,7,short,10,MO
3,2019-10-05,WN,WN,WN,4755,WN,N452WN,4755,14107,PHX,"Phoenix, AZ",14570,RNO,"Reno, NV",1755,21.0,6.0,2.0,1935,13.0,100.0,601.0,10,short,19,AZ
4,2018-05-11,WN,WN,WN,1002,WN,N706SW,1002,11884,GEG,"Spokane, WA",13796,OAK,"Oakland, CA",515,1.0,9.0,5.0,720,-14.0,125.0,723.0,5,medium,7,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99993,2018-12-15,AA,AA,AA,564,AA,N549UW,564,14747,SEA,"Seattle, WA",14100,PHL,"Philadelphia, PA",2226,-8.0,18.0,8.0,627,-2.0,301.0,2378.0,12,medium,6,WA
99994,2018-04-08,UA,UA_CODESHARE,UA,6054,YV,N86324,6054,12266,IAH,"Houston, TX",13256,MFE,"Mission/McAllen/Edinburg, TX",735,76.0,13.0,3.0,858,62.0,83.0,316.0,4,short,8,TX
99995,2019-02-01,AA,AA_CODESHARE,AA,4357,YX,N437YX,4357,10868,CAE,"Columbia, SC",11057,CLT,"Charlotte, NC",528,-4.0,14.0,10.0,626,-11.0,58.0,88.0,2,short,6,SC
99996,2018-07-08,AS,AS,AS,803,AS,N534AS,803,14831,SJC,"San Jose, CA",13830,OGG,"Kahului, HI",800,0.0,12.0,13.0,1022,1.0,322.0,2355.0,7,medium,10,CA


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [110]:
flight_model_df = flight_df[['dest','month','arr_delay','distance','state/country']]
cat_col = ['dest','month','state/country']
num_col = ['distance']

In [111]:
dummies = pd.get_dummies(flight_model_df,columns = cat_col)

In [112]:
dummies.dropna(inplace=True)

In [113]:
dummies

,arr_delay,distance,dest_ABE,dest_ABI,dest_ABQ,dest_ABR,dest_ABY,dest_ACK,dest_ACT,dest_ACV,dest_ACY,dest_ADK,dest_ADQ,dest_AEX,dest_AGS,dest_AKN,dest_ALB,dest_ALO,dest_ALW,dest_AMA,dest_ANC,dest_APN,dest_ART,dest_ASE,dest_ATL,dest_ATW,dest_ATY,dest_AUS,dest_AVL,dest_AVP,dest_AZA,dest_AZO,dest_BDL,dest_BET,dest_BFF,dest_BFL,dest_BFM,dest_BGM,dest_BGR,dest_BHM,dest_BIL,dest_BIS,dest_BJI,dest_BKG,dest_BLI,dest_BLV,dest_BMI,dest_BNA,dest_BOI,dest_BOS,dest_BPT,dest_BQK,dest_BQN,dest_BRD,dest_BRO,dest_BRW,dest_BTM,dest_BTR,dest_BTV,dest_BUF,dest_BUR,dest_BWI,dest_BZN,dest_CAE,dest_CAK,dest_CDC,dest_CDV,dest_CGI,dest_CHA,dest_CHO,dest_CHS,dest_CID,dest_CIU,dest_CKB,dest_CLE,dest_CLL,dest_CLT,dest_CMH,dest_CMI,dest_CMX,dest_CNY,dest_COD,dest_COS,dest_COU,dest_CPR,dest_CRP,dest_CRW,dest_CSG,dest_CVG,dest_CWA,dest_CYS,dest_DAB,dest_DAL,dest_DAY,dest_DBQ,dest_DCA,dest_DEN,dest_DFW,dest_DHN,dest_DIK,dest_DLH,dest_DRO,dest_DRT,dest_DSM,dest_DTW,dest_DUT,dest_DVL,dest_EAR,dest_EAT,dest_EAU,dest_ECP,dest_EGE,dest_EKO,dest_ELM,dest_ELP,dest_ERI,dest_ESC,dest_EUG,dest_EVV,dest_EWN,dest_EWR,dest_EYW,dest_FAI,dest_FAR,dest_FAT,dest_FAY,dest_FCA,dest_FLG,dest_FLL,dest_FLO,dest_FNT,dest_FSD,dest_FSM,dest_FWA,dest_GCC,dest_GCK,dest_GEG,dest_GFK,dest_GGG,dest_GJT,dest_GNV,dest_GPT,dest_GRB,dest_GRI,dest_GRK,dest_GRR,dest_GSO,dest_GSP,dest_GST,dest_GTF,dest_GTR,dest_GUC,dest_GUM,dest_HDN,dest_HGR,dest_HHH,dest_HIB,dest_HLN,dest_HNL,dest_HOB,dest_HOU,dest_HPN,dest_HRL,dest_HSV,dest_HTS,dest_HVN,dest_HYA,dest_HYS,dest_IAD,dest_IAG,dest_IAH,dest_ICT,dest_IDA,dest_ILM,dest_IMT,dest_IND,dest_INL,dest_IPT,dest_ISN,dest_ISP,dest_ITH,dest_ITO,dest_JAC,dest_JAN,dest_JAX,dest_JFK,dest_JHM,dest_JLN,dest_JMS,dest_JNU,dest_KOA,dest_KTN,dest_LAN,dest_LAR,dest_LAS,dest_LAW,dest_LAX,dest_LBB,dest_LBE,dest_LBF,dest_LBL,dest_LCH,dest_LCK,dest_LEX,dest_LFT,dest_LGA,dest_LGB,dest_LIH,dest_LIT,dest_LNK,dest_LNY,dest_LRD,dest_LSE,dest_LWB,dest_LWS,dest_LYH,dest_MAF,dest_MBS,dest_MCI,dest_MCO,dest_MDT,dest_MDW,dest_MEI,dest_MEM,dest_MFE,dest_MFR,dest_MGM,dest_MHK,dest_MHT,dest_MIA,dest_MKE,dest_MKG,dest_MKK,dest_MLB,dest_MLI,dest_MLU,dest_MMH,dest_MOB,dest_MOT,dest_MQT,dest_MRY,dest_MSN,dest_MSO,dest_MSP,dest_MSY,dest_MTJ,dest_MVY,dest_MYR,dest_OAJ,dest_OAK,dest_OGG,dest_OGS,dest_OKC,dest_OMA,dest_OME,dest_ONT,dest_ORD,dest_ORF,dest_ORH,dest_OTH,dest_OTZ,dest_OWB,dest_PAE,dest_PAH,dest_PBG,dest_PBI,dest_PDX,dest_PGD,dest_PGV,dest_PHF,dest_PHL,dest_PHX,dest_PIA,dest_PIB,dest_PIE,dest_PIH,dest_PIR,dest_PIT,dest_PLN,dest_PNS,dest_PPG,dest_PQI,dest_PRC,dest_PSC,dest_PSE,dest_PSG,dest_PSM,dest_PSP,dest_PUB,dest_PUW,dest_PVD,dest_PVU,dest_PWM,dest_RAP,dest_RDD,dest_RDM,dest_RDU,dest_RFD,dest_RHI,dest_RIC,dest_RKS,dest_RNO,dest_ROA,dest_ROC,dest_ROW,dest_RST,dest_RSW,dest_SAF,dest_SAN,dest_SAT,dest_SAV,dest_SBA,dest_SBN,dest_SBP,dest_SBY,dest_SCC,dest_SCE,dest_SCK,dest_SDF,dest_SEA,dest_SFB,dest_SFO,dest_SGF,dest_SGU,dest_SHD,dest_SHV,dest_SIT,dest_SJC,dest_SJT,dest_SJU,dest_SLC,dest_SLN,dest_SMF,dest_SMX,dest_SNA,dest_SPI,dest_SPN,dest_SPS,dest_SRQ,dest_STL,dest_STS,dest_STT,dest_STX,dest_SUN,dest_SUX,dest_SWF,dest_SWO,dest_SYR,dest_TLH,dest_TOL,dest_TPA,dest_TRI,dest_TTN,dest_TUL,dest_TUS,dest_TVC,dest_TWF,dest_TXK,dest_TYR,dest_TYS,dest_UIN,dest_USA,dest_VEL,dest_VLD,dest_VPS,dest_WRG,dest_WYS,dest_XNA,dest_XWA,dest_YAK,dest_YKM,dest_YUM,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,state/country_AK,state/country_AL,state/country_AR,state/country_AZ,state/country_CA,state/country_CO,state/country_CT,state/country_DC,state/country_FL,state/country_GA,state/country_HI,state/country_IA,state/country_ID,state/country_IL,state/country_IN,state/country_KS,state/country_KY,state/country_LA,state/country_MA,state/country_MD,state/country_ME,state/country_MI,state/country_MN,state/country_MO,state/country_MS,state/country_MT,state/country_NC,state/country_ND,state/country_NE,state/country_NH,state/country_NJ,state/country_NM,state/c

In [114]:
y_target = dummies[['arr_delay']].copy()

In [115]:
y_target

,arr_delay
0,-15.0
1,-16.0
2,-8.0
3,13.0
4,-14.0
...,...
99993,-2.0
99994,62.0
99995,-11.0
99996,1.0


In [116]:
dummies.drop('arr_delay',axis=1,inplace=True)

In [117]:
dummies

,distance,dest_ABE,dest_ABI,dest_ABQ,dest_ABR,dest_ABY,dest_ACK,dest_ACT,dest_ACV,dest_ACY,dest_ADK,dest_ADQ,dest_AEX,dest_AGS,dest_AKN,dest_ALB,dest_ALO,dest_ALW,dest_AMA,dest_ANC,dest_APN,dest_ART,dest_ASE,dest_ATL,dest_ATW,dest_ATY,dest_AUS,dest_AVL,dest_AVP,dest_AZA,dest_AZO,dest_BDL,dest_BET,dest_BFF,dest_BFL,dest_BFM,dest_BGM,dest_BGR,dest_BHM,dest_BIL,dest_BIS,dest_BJI,dest_BKG,dest_BLI,dest_BLV,dest_BMI,dest_BNA,dest_BOI,dest_BOS,dest_BPT,dest_BQK,dest_BQN,dest_BRD,dest_BRO,dest_BRW,dest_BTM,dest_BTR,dest_BTV,dest_BUF,dest_BUR,dest_BWI,dest_BZN,dest_CAE,dest_CAK,dest_CDC,dest_CDV,dest_CGI,dest_CHA,dest_CHO,dest_CHS,dest_CID,dest_CIU,dest_CKB,dest_CLE,dest_CLL,dest_CLT,dest_CMH,dest_CMI,dest_CMX,dest_CNY,dest_COD,dest_COS,dest_COU,dest_CPR,dest_CRP,dest_CRW,dest_CSG,dest_CVG,dest_CWA,dest_CYS,dest_DAB,dest_DAL,dest_DAY,dest_DBQ,dest_DCA,dest_DEN,dest_DFW,dest_DHN,dest_DIK,dest_DLH,dest_DRO,dest_DRT,dest_DSM,dest_DTW,dest_DUT,dest_DVL,dest_EAR,dest_EAT,dest_EAU,dest_ECP,dest_EGE,dest_EKO,dest_ELM,dest_ELP,dest_ERI,dest_ESC,dest_EUG,dest_EVV,dest_EWN,dest_EWR,dest_EYW,dest_FAI,dest_FAR,dest_FAT,dest_FAY,dest_FCA,dest_FLG,dest_FLL,dest_FLO,dest_FNT,dest_FSD,dest_FSM,dest_FWA,dest_GCC,dest_GCK,dest_GEG,dest_GFK,dest_GGG,dest_GJT,dest_GNV,dest_GPT,dest_GRB,dest_GRI,dest_GRK,dest_GRR,dest_GSO,dest_GSP,dest_GST,dest_GTF,dest_GTR,dest_GUC,dest_GUM,dest_HDN,dest_HGR,dest_HHH,dest_HIB,dest_HLN,dest_HNL,dest_HOB,dest_HOU,dest_HPN,dest_HRL,dest_HSV,dest_HTS,dest_HVN,dest_HYA,dest_HYS,dest_IAD,dest_IAG,dest_IAH,dest_ICT,dest_IDA,dest_ILM,dest_IMT,dest_IND,dest_INL,dest_IPT,dest_ISN,dest_ISP,dest_ITH,dest_ITO,dest_JAC,dest_JAN,dest_JAX,dest_JFK,dest_JHM,dest_JLN,dest_JMS,dest_JNU,dest_KOA,dest_KTN,dest_LAN,dest_LAR,dest_LAS,dest_LAW,dest_LAX,dest_LBB,dest_LBE,dest_LBF,dest_LBL,dest_LCH,dest_LCK,dest_LEX,dest_LFT,dest_LGA,dest_LGB,dest_LIH,dest_LIT,dest_LNK,dest_LNY,dest_LRD,dest_LSE,dest_LWB,dest_LWS,dest_LYH,dest_MAF,dest_MBS,dest_MCI,dest_MCO,dest_MDT,dest_MDW,dest_MEI,dest_MEM,dest_MFE,dest_MFR,dest_MGM,dest_MHK,dest_MHT,dest_MIA,dest_MKE,dest_MKG,dest_MKK,dest_MLB,dest_MLI,dest_MLU,dest_MMH,dest_MOB,dest_MOT,dest_MQT,dest_MRY,dest_MSN,dest_MSO,dest_MSP,dest_MSY,dest_MTJ,dest_MVY,dest_MYR,dest_OAJ,dest_OAK,dest_OGG,dest_OGS,dest_OKC,dest_OMA,dest_OME,dest_ONT,dest_ORD,dest_ORF,dest_ORH,dest_OTH,dest_OTZ,dest_OWB,dest_PAE,dest_PAH,dest_PBG,dest_PBI,dest_PDX,dest_PGD,dest_PGV,dest_PHF,dest_PHL,dest_PHX,dest_PIA,dest_PIB,dest_PIE,dest_PIH,dest_PIR,dest_PIT,dest_PLN,dest_PNS,dest_PPG,dest_PQI,dest_PRC,dest_PSC,dest_PSE,dest_PSG,dest_PSM,dest_PSP,dest_PUB,dest_PUW,dest_PVD,dest_PVU,dest_PWM,dest_RAP,dest_RDD,dest_RDM,dest_RDU,dest_RFD,dest_RHI,dest_RIC,dest_RKS,dest_RNO,dest_ROA,dest_ROC,dest_ROW,dest_RST,dest_RSW,dest_SAF,dest_SAN,dest_SAT,dest_SAV,dest_SBA,dest_SBN,dest_SBP,dest_SBY,dest_SCC,dest_SCE,dest_SCK,dest_SDF,dest_SEA,dest_SFB,dest_SFO,dest_SGF,dest_SGU,dest_SHD,dest_SHV,dest_SIT,dest_SJC,dest_SJT,dest_SJU,dest_SLC,dest_SLN,dest_SMF,dest_SMX,dest_SNA,dest_SPI,dest_SPN,dest_SPS,dest_SRQ,dest_STL,dest_STS,dest_STT,dest_STX,dest_SUN,dest_SUX,dest_SWF,dest_SWO,dest_SYR,dest_TLH,dest_TOL,dest_TPA,dest_TRI,dest_TTN,dest_TUL,dest_TUS,dest_TVC,dest_TWF,dest_TXK,dest_TYR,dest_TYS,dest_UIN,dest_USA,dest_VEL,dest_VLD,dest_VPS,dest_WRG,dest_WYS,dest_XNA,dest_XWA,dest_YAK,dest_YKM,dest_YUM,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,state/country_AK,state/country_AL,state/country_AR,state/country_AZ,state/country_CA,state/country_CO,state/country_CT,state/country_DC,state/country_FL,state/country_GA,state/country_HI,state/country_IA,state/country_ID,state/country_IL,state/country_IN,state/country_KS,state/country_KY,state/country_LA,state/country_MA,state/country_MD,state/country_ME,state/country_MI,state/country_MN,state/country_MO,state/country_MS,state/country_MT,state/country_NC,state/country_ND,state/country_NE,state/country_NH,state/country_NJ,state/country_NM,state/country_NV,

In [118]:
x_train,x_test,y_train,y_test = train_test_split(dummies, y_target ,test_size=0.3)

In [119]:
x_test

,distance,dest_ABE,dest_ABI,dest_ABQ,dest_ABR,dest_ABY,dest_ACK,dest_ACT,dest_ACV,dest_ACY,dest_ADK,dest_ADQ,dest_AEX,dest_AGS,dest_AKN,dest_ALB,dest_ALO,dest_ALW,dest_AMA,dest_ANC,dest_APN,dest_ART,dest_ASE,dest_ATL,dest_ATW,dest_ATY,dest_AUS,dest_AVL,dest_AVP,dest_AZA,dest_AZO,dest_BDL,dest_BET,dest_BFF,dest_BFL,dest_BFM,dest_BGM,dest_BGR,dest_BHM,dest_BIL,dest_BIS,dest_BJI,dest_BKG,dest_BLI,dest_BLV,dest_BMI,dest_BNA,dest_BOI,dest_BOS,dest_BPT,dest_BQK,dest_BQN,dest_BRD,dest_BRO,dest_BRW,dest_BTM,dest_BTR,dest_BTV,dest_BUF,dest_BUR,dest_BWI,dest_BZN,dest_CAE,dest_CAK,dest_CDC,dest_CDV,dest_CGI,dest_CHA,dest_CHO,dest_CHS,dest_CID,dest_CIU,dest_CKB,dest_CLE,dest_CLL,dest_CLT,dest_CMH,dest_CMI,dest_CMX,dest_CNY,dest_COD,dest_COS,dest_COU,dest_CPR,dest_CRP,dest_CRW,dest_CSG,dest_CVG,dest_CWA,dest_CYS,dest_DAB,dest_DAL,dest_DAY,dest_DBQ,dest_DCA,dest_DEN,dest_DFW,dest_DHN,dest_DIK,dest_DLH,dest_DRO,dest_DRT,dest_DSM,dest_DTW,dest_DUT,dest_DVL,dest_EAR,dest_EAT,dest_EAU,dest_ECP,dest_EGE,dest_EKO,dest_ELM,dest_ELP,dest_ERI,dest_ESC,dest_EUG,dest_EVV,dest_EWN,dest_EWR,dest_EYW,dest_FAI,dest_FAR,dest_FAT,dest_FAY,dest_FCA,dest_FLG,dest_FLL,dest_FLO,dest_FNT,dest_FSD,dest_FSM,dest_FWA,dest_GCC,dest_GCK,dest_GEG,dest_GFK,dest_GGG,dest_GJT,dest_GNV,dest_GPT,dest_GRB,dest_GRI,dest_GRK,dest_GRR,dest_GSO,dest_GSP,dest_GST,dest_GTF,dest_GTR,dest_GUC,dest_GUM,dest_HDN,dest_HGR,dest_HHH,dest_HIB,dest_HLN,dest_HNL,dest_HOB,dest_HOU,dest_HPN,dest_HRL,dest_HSV,dest_HTS,dest_HVN,dest_HYA,dest_HYS,dest_IAD,dest_IAG,dest_IAH,dest_ICT,dest_IDA,dest_ILM,dest_IMT,dest_IND,dest_INL,dest_IPT,dest_ISN,dest_ISP,dest_ITH,dest_ITO,dest_JAC,dest_JAN,dest_JAX,dest_JFK,dest_JHM,dest_JLN,dest_JMS,dest_JNU,dest_KOA,dest_KTN,dest_LAN,dest_LAR,dest_LAS,dest_LAW,dest_LAX,dest_LBB,dest_LBE,dest_LBF,dest_LBL,dest_LCH,dest_LCK,dest_LEX,dest_LFT,dest_LGA,dest_LGB,dest_LIH,dest_LIT,dest_LNK,dest_LNY,dest_LRD,dest_LSE,dest_LWB,dest_LWS,dest_LYH,dest_MAF,dest_MBS,dest_MCI,dest_MCO,dest_MDT,dest_MDW,dest_MEI,dest_MEM,dest_MFE,dest_MFR,dest_MGM,dest_MHK,dest_MHT,dest_MIA,dest_MKE,dest_MKG,dest_MKK,dest_MLB,dest_MLI,dest_MLU,dest_MMH,dest_MOB,dest_MOT,dest_MQT,dest_MRY,dest_MSN,dest_MSO,dest_MSP,dest_MSY,dest_MTJ,dest_MVY,dest_MYR,dest_OAJ,dest_OAK,dest_OGG,dest_OGS,dest_OKC,dest_OMA,dest_OME,dest_ONT,dest_ORD,dest_ORF,dest_ORH,dest_OTH,dest_OTZ,dest_OWB,dest_PAE,dest_PAH,dest_PBG,dest_PBI,dest_PDX,dest_PGD,dest_PGV,dest_PHF,dest_PHL,dest_PHX,dest_PIA,dest_PIB,dest_PIE,dest_PIH,dest_PIR,dest_PIT,dest_PLN,dest_PNS,dest_PPG,dest_PQI,dest_PRC,dest_PSC,dest_PSE,dest_PSG,dest_PSM,dest_PSP,dest_PUB,dest_PUW,dest_PVD,dest_PVU,dest_PWM,dest_RAP,dest_RDD,dest_RDM,dest_RDU,dest_RFD,dest_RHI,dest_RIC,dest_RKS,dest_RNO,dest_ROA,dest_ROC,dest_ROW,dest_RST,dest_RSW,dest_SAF,dest_SAN,dest_SAT,dest_SAV,dest_SBA,dest_SBN,dest_SBP,dest_SBY,dest_SCC,dest_SCE,dest_SCK,dest_SDF,dest_SEA,dest_SFB,dest_SFO,dest_SGF,dest_SGU,dest_SHD,dest_SHV,dest_SIT,dest_SJC,dest_SJT,dest_SJU,dest_SLC,dest_SLN,dest_SMF,dest_SMX,dest_SNA,dest_SPI,dest_SPN,dest_SPS,dest_SRQ,dest_STL,dest_STS,dest_STT,dest_STX,dest_SUN,dest_SUX,dest_SWF,dest_SWO,dest_SYR,dest_TLH,dest_TOL,dest_TPA,dest_TRI,dest_TTN,dest_TUL,dest_TUS,dest_TVC,dest_TWF,dest_TXK,dest_TYR,dest_TYS,dest_UIN,dest_USA,dest_VEL,dest_VLD,dest_VPS,dest_WRG,dest_WYS,dest_XNA,dest_XWA,dest_YAK,dest_YKM,dest_YUM,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,state/country_AK,state/country_AL,state/country_AR,state/country_AZ,state/country_CA,state/country_CO,state/country_CT,state/country_DC,state/country_FL,state/country_GA,state/country_HI,state/country_IA,state/country_ID,state/country_IL,state/country_IN,state/country_KS,state/country_KY,state/country_LA,state/country_MA,state/country_MD,state/country_ME,state/country_MI,state/country_MN,state/country_MO,state/country_MS,state/country_MT,state/country_NC,state/country_ND,state/country_NE,state/country_NH,state/country_NJ,state/country_NM,state/country_NV,

In [120]:
X = scaler.fit_transform(x_train[num_col])

In [121]:
x_train['distance'] = X

In [122]:
x_train

,distance,dest_ABE,dest_ABI,dest_ABQ,dest_ABR,dest_ABY,dest_ACK,dest_ACT,dest_ACV,dest_ACY,dest_ADK,dest_ADQ,dest_AEX,dest_AGS,dest_AKN,dest_ALB,dest_ALO,dest_ALW,dest_AMA,dest_ANC,dest_APN,dest_ART,dest_ASE,dest_ATL,dest_ATW,dest_ATY,dest_AUS,dest_AVL,dest_AVP,dest_AZA,dest_AZO,dest_BDL,dest_BET,dest_BFF,dest_BFL,dest_BFM,dest_BGM,dest_BGR,dest_BHM,dest_BIL,dest_BIS,dest_BJI,dest_BKG,dest_BLI,dest_BLV,dest_BMI,dest_BNA,dest_BOI,dest_BOS,dest_BPT,dest_BQK,dest_BQN,dest_BRD,dest_BRO,dest_BRW,dest_BTM,dest_BTR,dest_BTV,dest_BUF,dest_BUR,dest_BWI,dest_BZN,dest_CAE,dest_CAK,dest_CDC,dest_CDV,dest_CGI,dest_CHA,dest_CHO,dest_CHS,dest_CID,dest_CIU,dest_CKB,dest_CLE,dest_CLL,dest_CLT,dest_CMH,dest_CMI,dest_CMX,dest_CNY,dest_COD,dest_COS,dest_COU,dest_CPR,dest_CRP,dest_CRW,dest_CSG,dest_CVG,dest_CWA,dest_CYS,dest_DAB,dest_DAL,dest_DAY,dest_DBQ,dest_DCA,dest_DEN,dest_DFW,dest_DHN,dest_DIK,dest_DLH,dest_DRO,dest_DRT,dest_DSM,dest_DTW,dest_DUT,dest_DVL,dest_EAR,dest_EAT,dest_EAU,dest_ECP,dest_EGE,dest_EKO,dest_ELM,dest_ELP,dest_ERI,dest_ESC,dest_EUG,dest_EVV,dest_EWN,dest_EWR,dest_EYW,dest_FAI,dest_FAR,dest_FAT,dest_FAY,dest_FCA,dest_FLG,dest_FLL,dest_FLO,dest_FNT,dest_FSD,dest_FSM,dest_FWA,dest_GCC,dest_GCK,dest_GEG,dest_GFK,dest_GGG,dest_GJT,dest_GNV,dest_GPT,dest_GRB,dest_GRI,dest_GRK,dest_GRR,dest_GSO,dest_GSP,dest_GST,dest_GTF,dest_GTR,dest_GUC,dest_GUM,dest_HDN,dest_HGR,dest_HHH,dest_HIB,dest_HLN,dest_HNL,dest_HOB,dest_HOU,dest_HPN,dest_HRL,dest_HSV,dest_HTS,dest_HVN,dest_HYA,dest_HYS,dest_IAD,dest_IAG,dest_IAH,dest_ICT,dest_IDA,dest_ILM,dest_IMT,dest_IND,dest_INL,dest_IPT,dest_ISN,dest_ISP,dest_ITH,dest_ITO,dest_JAC,dest_JAN,dest_JAX,dest_JFK,dest_JHM,dest_JLN,dest_JMS,dest_JNU,dest_KOA,dest_KTN,dest_LAN,dest_LAR,dest_LAS,dest_LAW,dest_LAX,dest_LBB,dest_LBE,dest_LBF,dest_LBL,dest_LCH,dest_LCK,dest_LEX,dest_LFT,dest_LGA,dest_LGB,dest_LIH,dest_LIT,dest_LNK,dest_LNY,dest_LRD,dest_LSE,dest_LWB,dest_LWS,dest_LYH,dest_MAF,dest_MBS,dest_MCI,dest_MCO,dest_MDT,dest_MDW,dest_MEI,dest_MEM,dest_MFE,dest_MFR,dest_MGM,dest_MHK,dest_MHT,dest_MIA,dest_MKE,dest_MKG,dest_MKK,dest_MLB,dest_MLI,dest_MLU,dest_MMH,dest_MOB,dest_MOT,dest_MQT,dest_MRY,dest_MSN,dest_MSO,dest_MSP,dest_MSY,dest_MTJ,dest_MVY,dest_MYR,dest_OAJ,dest_OAK,dest_OGG,dest_OGS,dest_OKC,dest_OMA,dest_OME,dest_ONT,dest_ORD,dest_ORF,dest_ORH,dest_OTH,dest_OTZ,dest_OWB,dest_PAE,dest_PAH,dest_PBG,dest_PBI,dest_PDX,dest_PGD,dest_PGV,dest_PHF,dest_PHL,dest_PHX,dest_PIA,dest_PIB,dest_PIE,dest_PIH,dest_PIR,dest_PIT,dest_PLN,dest_PNS,dest_PPG,dest_PQI,dest_PRC,dest_PSC,dest_PSE,dest_PSG,dest_PSM,dest_PSP,dest_PUB,dest_PUW,dest_PVD,dest_PVU,dest_PWM,dest_RAP,dest_RDD,dest_RDM,dest_RDU,dest_RFD,dest_RHI,dest_RIC,dest_RKS,dest_RNO,dest_ROA,dest_ROC,dest_ROW,dest_RST,dest_RSW,dest_SAF,dest_SAN,dest_SAT,dest_SAV,dest_SBA,dest_SBN,dest_SBP,dest_SBY,dest_SCC,dest_SCE,dest_SCK,dest_SDF,dest_SEA,dest_SFB,dest_SFO,dest_SGF,dest_SGU,dest_SHD,dest_SHV,dest_SIT,dest_SJC,dest_SJT,dest_SJU,dest_SLC,dest_SLN,dest_SMF,dest_SMX,dest_SNA,dest_SPI,dest_SPN,dest_SPS,dest_SRQ,dest_STL,dest_STS,dest_STT,dest_STX,dest_SUN,dest_SUX,dest_SWF,dest_SWO,dest_SYR,dest_TLH,dest_TOL,dest_TPA,dest_TRI,dest_TTN,dest_TUL,dest_TUS,dest_TVC,dest_TWF,dest_TXK,dest_TYR,dest_TYS,dest_UIN,dest_USA,dest_VEL,dest_VLD,dest_VPS,dest_WRG,dest_WYS,dest_XNA,dest_XWA,dest_YAK,dest_YKM,dest_YUM,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,state/country_AK,state/country_AL,state/country_AR,state/country_AZ,state/country_CA,state/country_CO,state/country_CT,state/country_DC,state/country_FL,state/country_GA,state/country_HI,state/country_IA,state/country_ID,state/country_IL,state/country_IN,state/country_KS,state/country_KY,state/country_LA,state/country_MA,state/country_MD,state/country_ME,state/country_MI,state/country_MN,state/country_MO,state/country_MS,state/country_MT,state/country_NC,state/country_ND,state/country_NE,state/country_NH,state/country_NJ,state/country_NM,state/country_NV,

In [125]:
clf = LinearRegression()
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [131]:
r2_score(y_test,y_pred)

-2.7774748791622464e+20

In [133]:
passenger_df

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,air_time,unique_carrier,airline_id,unique_carrier_name,region,carrier_name,origin_airport_id,origin_city_name,origin_country_name,dest_airport_id,dest_city_name,dest_country_name,aircraft_type,year,month,class
0,1.0,1.0,31200.0,156.0,133.0,0.0,0.0,1141.0,150.0,G4,20368,Allegiant Air,D,Allegiant Air,10408,"Appleton, WI",United States,14761,"Sanford, FL",United States,698,2018,11,F
1,16.0,16.0,736000.0,3680.0,2864.0,0.0,0.0,957.0,2111.0,F9,20436,Frontier Airlines Inc.,D,Frontier Airlines Inc.,13204,"Orlando, FL",United States,11433,"Detroit, MI",United States,699,2016,6,F
2,0.0,57.0,76950.0,342.0,157.0,4701.0,7618.0,24.0,717.0,H6,20336,Hageland Aviation Service,D,Hageland Aviation Service,10551,"Bethel, AK",United States,12831,"Kasigluk, AK",United States,35,2017,4,F
3,0.0,1.0,31841.0,58.0,43.0,0.0,0.0,440.0,0.0,AC,19531,Air Canada,I,Air Canada,11433,"Detroit, MI",United States,16149,"Ottawa, Canada",Canada,698,2018,3,L
4,4.0,3.0,57276.0,210.0,197.0,0.0,0.0,284.0,152.0,S5,20448,Shuttle America Corp.,D,Shuttle America Corp.,11618,"Newark, NJ",United States,13931,"Norfolk, VA",United States,677,2016,10,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,27.0,0.0,0.0,0.0,0.0,0.0,0.0,6745.0,0.0,DL,19790,Delta Air Lines Inc.,P,Delta Air Lines Inc.,13744,"Tokyo, Japan",Japan,12478,"New York, NY",United States,650,2015,10,F
99996,163.0,154.0,2075500.0,7700.0,5343.0,0.0,0.0,383.0,10100.0,EV,20366,ExpressJet Airlines LLC,D,ExpressJet Airlines Inc.,10781,"Baton Rouge, LA",United States,11298,"Dallas/Fort Worth, TX",United States,629,2015,3,F
99997,0.0,1.0,3400.0,9.0,0.0,0.0,0.0,67.0,28.0,H6,20336,Hageland Aviation Service,D,Hageland Aviation Service,11535,"Elim, AK",United States,15478,"Unalakleet, AK",United States,416,2018,9,F
99998,28.0,27.0,3294000.0,6533.0,4830.0,654319.0,0.0,3434.0,12215.0,AA,19805,American Airlines Inc.,A,American Airlines Inc.,13165,"Manchester, United Kingdom",United Kingdom,14100,"Philadelphia, PA",United States,696,2019,3,F


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.